# get image description from moondream
save iamge_name, prompt and output text to a csv file

In [1]:
import torch
import argparse
from PIL import Image
from moondream import Moondream, detect_device
from moondream.vision_encoder import VisionEncoder
from queue import Queue
from threading import Thread
from transformers import (
    TextIteratorStreamer,
    CodeGenTokenizerFast as Tokenizer,
)
import re
import os
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

/Users/xing.zhang/anaconda3/envs/moondream/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/xing.zhang/anaconda3/envs/moondream/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/xing.zhang/anaconda3/envs/moondream/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/xing.zhang/anaconda3/envs/moondream/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is depr

## load model

In [2]:
device, dtype = detect_device()
# print(device, dtype)

model_id = "vikhyatk/moondream1"
tokenizer = Tokenizer.from_pretrained(model_id)
moondream = Moondream.from_pretrained(model_id).to(device=device, dtype=dtype)
moondream.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Moondream(
  (vision_encoder): VisionEncoder(
    (encoder): ModelHolder(
      (model): VisualHolder(
        (visual): VisionTransformer(
          (patch_embed): LinearPatchEmbedding(
            (linear): Linear(in_features=588, out_features=1152, bias=True)
          )
          (pos_drop): Dropout(p=0.0, inplace=False)
          (patch_drop): Identity()
          (norm_pre): Identity()
          (blocks): Sequential(
            (0): Block(
              (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (attn): Attention(
                (qkv): Linear(in_features=1152, out_features=3456, bias=True)
                (q_norm): Identity()
                (k_norm): Identity()
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=1152, out_features=1152, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
              )
              (ls1): Identity()
              (drop_path1): Identity()
  

## make prediction

In [3]:
repo = 'rug_images'
images = os.listdir(repo)
print(f"# of rug images is {len(images)}")

# prompt = """
# Assuming you are a store assistant please analyze the item in the picture.
#  Only return the data in a json with the keys shape, dominant_colors, style, description, material, suggested_title, theme. 
#  Please restrict the product description to 100 words. 
#  For theme please return any special characteristics like nature, geometric, abstract etc. 
# """

prompt = """describe the colors and patterns"""

answers, names = [], []
for name in os.listdir(repo):
    image = Image.open(repo + '/' + name)
    image_embeds = moondream.encode_image(image) # VisionEncoder()(image).to(device, dtype)
    answer = moondream.answer_question( image_embeds,prompt,  tokenizer)

    answers.append(answer)
    names.append(name)

df_moondream = pd.DataFrame({'image': names, 'prompt': [prompt]*len(names), 'answer': answers})
df_moondream.head(2)

# of rug images is 183


,image,prompt,answer
0,5031 – Metok Crimson Wool & Silk.jpg,describe the colors and patterns,"The rug in the image is red with a predominantly white and gray color scheme. It features a floral pattern, with various flowers and plants depicted on the red background. The rug also includes some gray elements, adding contrast to the overall design. The pattern is intricate and visually appealing, making it an attractive decorative piece for the room."
1,6103-Over Dyed -10×13.5.jpg,describe the colors and patterns,"The purple carpet features a bold, dark purple color with some lighter shades, creating a striking contrast. The design on the carpet consists of intricate patterns and shapes, which contribute to its unique and eye-catching appearance. The patterns are likely to be the main focus of the carpet and can be seen in various parts of the rug, from the center to the edges."


In [4]:
df_moondream.to_csv('description_by_moondream1.csv', index=False)